# Notebook showcasing use of Google Cloud GenAI(Palm API) + Doc AI to extract information from documents

In [ ]:
#install dependencies
!pip install google-cloud-aiplatform --upgrade
#!apt-get install poppler-utils
!pip install google-cloud-core
!pip install google-cloud-documentai
!pip install google-cloud-storage
!pip install simplejson
#!pip install pdf2image

In [1]:
#import libraries
from google.api_core.client_options import ClientOptions
from google.cloud import documentai
from PIL import Image, ImageDraw
import os
import pandas as pd
from IPython.display import display, Image
from pdf2image import convert_from_path, convert_from_bytes

In [ ]:
### Authenti

In [ ]:
#Run only if using Google Colab Notebooks
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [3]:
file_path='./sample_data/US_PTO_Sample.pdf'

In [8]:
from IPython.display import IFrame
IFrame(file_path, width=800, height=600)

In [12]:
def process_document_sample(
    project_id: str,
    location: str,
    processor_id: str,
    file_path: str,
    mime_type: str,
    field_mask: str = None,
):

    client = documentai.DocumentProcessorServiceClient()

    name = client.processor_path(project_id, location, processor_id)

    # Import the file into memory
    with open(file_path, "rb") as image:
        image_content = image.read()

    # Load the image content
    raw_document = documentai.RawDocument(content=image_content, mime_type=mime_type)


    request = documentai.ProcessRequest(
        name=name, raw_document=raw_document
    )

    result = client.process_document(request=request)


    document = result.document

    # Read the text recognition output from the processor
    return(document.text)



In [13]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

In [14]:
#For PDF Docs
ocr_output = process_document_sample(
  project_id="398507275014",
  location="us",
  processor_id="2fb6b1be15c7f2d",
    mime_type = 'application/pdf',
    field_mask = None,
  file_path= file_path
#"./genai_demo_data/demo_data.pdf"
)

In [ ]:
#For TIFF docs
ocr_output = process_document_sample(
  project_id="398507275014",
  location="us",
  processor_id="2fb6b1be15c7f2d",
    mime_type = 'image/tiff',
    field_mask = None,
  file_path="./genai_demo_data/demo_data.tiff"
)

In [9]:
print(ocr_output[:1000])

NameError: name 'ocr_output' is not defined

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#persistent_path = '/content/drive/Shareddrives/AIF LLM Demos/BigQuery'

In [13]:
#from google.colab import auth as google_auth
#google_auth.authenticate_user()

from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

def predict_large_language_model_sample(
    api_endpoint: str,
    project: str,
    endpoint_id: str,
    content: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    location: str = "us-central1",
):
  # The AI Platform services require regional API endpoints.
  client_options = {"api_endpoint": api_endpoint}
  # Initialize client that will be used to create and send requests.
  # This client only needs to be created once, and can be reused for multiple requests.
  client = aiplatform.gapic.PredictionServiceClient(
      client_options=client_options
  )
  instance_dict = {"content": content}
  instance = json_format.ParseDict(instance_dict, Value())
  instances = [instance]
  parameters_dict = {
      "temperature": temperature,
      "maxDecodeSteps": max_decode_steps,
      "topP": top_p,
      "topK": top_k,
  }
  parameters = json_format.ParseDict(parameters_dict, Value())
  endpoint = client.endpoint_path(
      project=project, location=location, endpoint=endpoint_id
  )
  response = client.predict(
      endpoint=endpoint, instances=instances, parameters=parameters
  )
  #print("response")
  predictions = response.predictions
  for prediction in predictions:
    print(" prediction:", dict(prediction))
    print(dict(prediction))
  return(dict(prediction))



In [ ]:
llm_output = predict_large_language_model_sample("us-central1-aiplatform.googleapis.com", "cloud-large-language-models", "4511608470067216384", '''"text": "RECORDING REQUESTED BY:\\nEscrow Technologies Inc.\\nOrder No. 1255381 D\\nEscrow No. N13-00200-TE\\nParcel No. 449-362-30\\nAND WHEN RECORDED MAIL TO:\\nJASMEET S. BHATIA\\n3972 BANYAN STREET\\nIRVINECA 92606\\nGRANT DEED\\nTHE UNDERSIGNED GRANTOR(S) DECLARE(S) THAT DOCUMENTARY TRANSFER TAX IS $881.10\\nX computed on full value of property conveyed, or\\ncomputed on full value less liens or encumbrances remaining at the time of sale.\\nunincorporated area:\\nX City of:\\nIrvine\\nFOR A VALUABLE CONSIDERATION, receipt of which is hereby acknowledged,\\n807 South Valley, LLC\\n}\\n}\\nRecorded in Official Records, Orange County\\nHugh Nguyen, Clerk-Recorder\\nhereby GRANT(S) to Jasmeet S. Bhatia and Shefali Dua, Husband And Wife As Community Property with\\nright of survivorship\\nthe following described real property in the City of Irvine, County of Orange, State of California:\\nIN BOOK 299\\nLot 63 of Tract no. 7464, in the City of Irvine, County of Orange, State of California, as per map recored\\nPAGES 24 to 30 of miscellaneous maps, in the office of the County Recorder of said County.\\nDate September 19, 2013\\n807 South Valley, LLC\\nS.S.\\n* $ RO 0 0 6 2 1 3 5 0 1 $ *\\n2013000551976 9:50 am 09/25/13\\n276 404 G02 2 26\\n440.55 440.55 0.00 0.00 3.00 0.00 0.00 0.00\\nSPACE ABOVE THIS LINE FOR RECORDER\'S USE\\nMansoori\\nBy: Authorized Signor\\nSTATE OF CALIFORNIA\\nCOUNTY OF Orange\\nOn September 20, 2013 before me,\\nT Eden\\na notary public in and for said State, personally appeared Kamran Mansoori\\nwho proved to me on the basis of satisfactory evidence to be the person(s) whose name(s) is/are subscribed to the\\nwithin instrument and acknowledged to me that he/she/they executed the same in his/her/their authorized\\ncapacity(ies), and that by his/her/their signature(s) on the instrument the person(s), or the entity upon behalf of\\nwhich the person(s) acted, executed the instrument.\\n-9\\nWITNESS my hand and official sean\\nHap\\nSignature\\n(Seal)\\nRem Dala\\nAuthorized Signor\\nI certify under PENALTY OF PERJURY under the laws of the State of California that the foregoing paragraph is\\ntrue and correct.\\nsuber\\n9.00\\nm\\nT. EDEN\\nCommission # 1878759\\nNotary Public - California\\nOrange County\\nMy Comm. Expires Feb 6, 2014\\nMail Tax Statement to: SAME AS ABOVE or Address Noted Below\\nSTATE OF CALIFORNIA\\nCOUNTY OF Orange\\nOn September 20, 2013, before me,\\nT. Eden\\na notary public in and for said State, personally appeared Fahimi Rashid\\nwho proved to me on the basis of satisfactory evidence to be the person(s) whose name(s) is/are subscribed to the within\\ninstrument and acknowledged to me that he/she/they executed the same in his/her/their authorized capacity(ies), and that by\\nhis/her/their signature(s) on the instrument the person(s), or the entity upon behalf of which the person(s) acted, executed the\\ninstrument.\\nI certify under PENALTY OF PERJURY under the laws of the State of California that the foregoing paragraph is true and\\ncorrect.\\nWITNESS my hand and official seal,\\nSignature\\nCALIFORNIA ALL-PURPOSE ACKNOWLEDGEMENT\\n}\\n} S.S.\\nDESCRIPTION OF ATTACHED DOCUMENT\\nTitle or Type of Document: Grant Deed\\nDocument Date: September 20, 2013.\\nSigner(s) other than named above Kamran Mansoori\\nCORPORATE OFFICER(S)\\nTITLE(S)\\nPARTNER(S)-\\nCAPACITY (IES) CLAIMED BY SIGNER(S)\\nINDIVIDUAL\\nATTORNEY-IN-FACT\\nTRUSTEE(S)\\nOTHER\\nOPTIONAL\\nThough the information below is not required by law, it may prove valuable to persons relying\\non the document and could prevent fraudulent removal and reattachment of this form to another document.\\nLIMITED\\nGENERAL\\nGUARDIAN OR CONSERVATOR\\n(Seal)\\nSIGNER IS REPRESENTING:\\nRight Thumbprint\\nOf Signer\\nTop of thumb here\\nCLOT\\nNumber of Pages:\\n☐INDIVIDUAL\\none\\nT. EDEN\\nCommission # 1878759\\nNotary Public - California\\nOrange County\\nMy Comm. Expires Feb 6, 2014\\nCORPORATE OFFICER(S)\\nTITLE(S)\\nPARTNER(S) ☐ LIMITED\\nGENERAL\\n☐TRUSTEE(S)\\nATTORNEY-IN-FACT\\nOTHER\\nGUARDIAN OR CONSERVATOR\\nRight Thumbprint\\nof Signer\\nTop of thumb here\\nORANGE\\nORANGE COUNTY\\nHALL OF RECORDS AND FINANCE\\n12 CIVIC CENTER PLAZA, ROOM 101, P.O. BOX 238, SANTA ANA, CALIFORNIA 92702-0238\\nPHONE (714) 834-2500 FAX (714) 834-2675\\n807 South Valley LLC\\n3972 Banyan St\\nIrvine, CA 92606-2630\\nCLERK-RECORDER\\nDear Valued Customer,\\nWWW.OCRECORDER.COM\\nCOURTESY NOTICE\\nSUBJECT: GRANT DEED RECORDING CONFIRMATION\\nOctober 8, 2013\\nThe Orange County Clerk-Recorder Department is working to protect our customers from\\npotential fraudulent and illegal activities.\\nOne step we\'re taking is to send out courtesy notices regarding recent property recording\\ntransactions. This is to ensure legitimacy and accuracy.\\nOur records indicate that a grant deed was recently recorded under your name in our office.\\nPlease find enclosed a copy of the grant deed for your reference.\\nIF THE ABOVE TRANSACTION IS CORRECT, NO ACTION ON YOUR PART IS\\nREQUIRED.\\nIf you are unaware of this recording, please call your local law enforcement agency as soon as\\npossible to further investigate.\\nIf you have any questions, please contact the Clerk-Recorder Department at (714) 834-2500. For\\nmore information regarding real property fraud, please read the enclosed blue sheet, or visit us\\nonline at www.ocrecorder.com.\\nThank you for your business and our department looks forward to serving you again.\\n"

Give me a bulleted list of following information extracted form text above:
- Name of Seller 1
- Seller 1 Type
-  If seller is an LLC, then name of the officer of the LLC
- Name of Seller 2
- Name of buyer 1
- Name of buyer 2
- Name of buyer 3
- Type of ownership
- Name of Title Company
- Address of the property
- Title Order number
- Document transfer tax
- List of parcels
- Property details''', 0.2, 256, 0.8, 40, "us-central1")

In [4]:
#print(llm_output['content'])

In [16]:
prompt_suffix = '''What is the name of the inventor(s)?'''



In [17]:
ocr_text = ocr_output+prompt_suffix

ocr_text


'(12) United States Patent\nLethin et al.\n(54) METHODS AND APPARATUS FOR LOCAL\nMEMORY COMPACTION\n(75) Inventors: Richard A. Lethin, New York, NY (US);\nAllen K. Leung, New York, NY (US);\nBenoit J. Meister, New York, NY (US);\nNicolas T. Vasilache, New York, NY\n(US); David E. Wohlford, Portland, OR\n(US)\n(73) Assignee: Reservoir Labs, Inc., New York, NY\n(US)\n(*) Notice:\n(21) Appl. No.: 12/365,780\n(22) Filed:\nFeb. 4, 2009\n(65)\nPrior Publication Data\nUS 2010/0192138 A1 Jul. 29, 2010\nRelated U.S. Application Data\n(60) Provisional application No. 61/065,294, filed on Feb.\n8, 2008.\n(51) Int. Cl.\n(56)\nSubject to any disclaimer, the term of this\npatent is extended or adjusted under 35\nU.S.C. 154(b) by 1287 days.\nG06F 9/45\n(52) U.S. CI.\nUSPC\n(58)\n5,442,699 A\n5,442,797 A\n5,613,136 A\n(2006.01)\nField of Classification Search\nUSPC\nSee application file for complete search history.\nReferences Cited\nU.S. PATENT DOCUMENTS\n717/151\n8/1995 Arnold et al.\n8/1995 Casavan

In [20]:
llm_output = predict_large_language_model_sample("us-central1-aiplatform.googleapis.com", 
                                                 "cloud-large-language-models", 
                                                 "4511608470067216384", 
                                                 ocr_text, 
                                                 0.2, 256, 0.8, 40, 
                                                 "us-central1")

 prediction: {'safetyAttributes': <proto.marshal.collections.maps.MapComposite object at 0x7fc9dfeec310>, 'content': 'Seller 1: Reayoung, LLC, a New York Limited Liability Company\nSeller 1 Type: LLC\nSeller 1 Officer: Feng Pan\nSeller 2: \nBuyer 1: Jasmeet Bhatia and Shefali Dua, Husband and Wife as Community Property with Right of Survivorship\nBuyer 2: \nBuyer 3: \nType of ownership: Community Property with Right of Survivorship\nTitle Company: Fidelity National Title\nAddress of the property: 34 Lowland\nIrvine, CA 92602\nTitle Order number: 00371527-997-OC1-JA1\nDocument transfer tax: 3113.00\nList of parcels:\nParcel 1:\nLOT 1 OF TRACT NO. 16703, IN THE CITY OF IRVINE, COUNTY OF ORANGE, STATE OF CALIFORNIA, AS\nSHOWN ON A MAP RECORDED IN BOOK 921, PAGES 14 THROUGH 19, INCLUSIVE, OF MISCELLANEOUS\nMAPS IN THE OFFICE OF THE COUNTY RECORDER FOR ORANGE COUNTY, CALIFORNIA.'}
{'safetyAttributes': <proto.marshal.collections.maps.MapComposite object at 0x7fc9dfeefb50>, 'content': 'Seller

In [ ]:
llm_output = predict_large_language_model_sample("us-central1-aiplatform.googleapis.com", 
                                                 "cloud-large-language-models", 
                                                 "4511608470067216384", 
                                                 ocr_text, 
                                                 0.2, 256, 0.8, 40, 
                                                 "us-central1")

In [21]:
print(llm_output['content'])

Seller 1: Reayoung, LLC, a New York Limited Liability Company
Seller 1 Type: LLC
Seller 1 Officer: Feng Pan
Seller 2: 
Buyer 1: Jasmeet Bhatia and Shefali Dua, Husband and Wife as Community Property with Right of Survivorship
Buyer 2: 
Buyer 3: 
Type of ownership: Community Property with Right of Survivorship
Title Company: Fidelity National Title
Address of the property: 34 Lowland
Irvine, CA 92602
Title Order number: 00371527-997-OC1-JA1
Document transfer tax: 3113.00
List of parcels:
Parcel 1:
LOT 1 OF TRACT NO. 16703, IN THE CITY OF IRVINE, COUNTY OF ORANGE, STATE OF CALIFORNIA, AS
SHOWN ON A MAP RECORDED IN BOOK 921, PAGES 14 THROUGH 19, INCLUSIVE, OF MISCELLANEOUS
MAPS IN THE OFFICE OF THE COUNTY RECORDER FOR ORANGE COUNTY, CALIFORNIA.


In [18]:
#from google.cloud import vertexai
import vertexai
from vertexai.preview.language_models import TextGenerationModel

def predict_large_language_model_sample(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
    ) :
    """Predict using a Large Language Model."""
    vertexai.init(project=project_id, location=location)
    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
      model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,)
    print(f"Response from Model: {response.text}")


In [20]:
predict_large_language_model_sample("jsb-alto", "text-bison@001", 0.2, 256, 0.8, 40, 
                                    ocr_text, 
                                    "us-central1")

Response from Model: Richard A. Lethin, Allen K. Leung, Benoit J. Meister, Nicolas T. Vasilache, David E. Wohlford
